In [4]:
import pandas as pd
import numpy as np
# import seaborn as sns

# Rodrigo deu essa ideia de boas práticas na limpesa dele e só aproveitei
PATH = 'seeders/olist_order_items_dataset.csv'
CLEAN_PATH = 'clean_datasets/order.csv'

# Deixei como df para padronizar os arquivos
df = pd.read_csv(PATH)

# df 
# display(df)

df.columns = ['id_pedido', 'numero_itens', 'id_produto', 'id_vendedor', 'limite_entrega', 'preco', 'frete_valor']
# df
df.dtypes
# Não existem incosistências com relação ao tipo da variável

df = df.drop_duplicates(subset=['id_pedido'], keep='last')
# Sempre que temos um numero de pedidos maior que 1, temos primeiro uma coluna com um numero_pedido = 1, e outra com o dado 'real'. Essa aparece por último
# Por isso, mantive a última, isso foi provado no código abaixo:
# display(df.loc[df['numero_itens'] > 1])

# display(df.loc[df['id_pedido'] == '0008288aa423d2a3f00fcb17cd7d8719'])

# Visualização de itens duplicados após o tratamento
df[df.duplicated(['id_pedido'], keep=False )]

df.isnull().sum() #nenhum valor nulo

# É neceessário excluir outliers em alguma coluna?
# -- em numero_itens; preco, e frete_valor (pela documentação, são dependentes umas das outras)
# display(df)

# display(df.loc[df['numero_itens'] > 1])

# estabelecendo o limite
limite = 1.5 * np.std(df[['preco', 'frete_valor']])

# visualização com linhas
for coluna in [ 'preco', 'frete_valor']:
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers_coluna = df[coluna][df[coluna] < limite_inferior].tolist()
    outliers_coluna.extend(df[coluna][df[coluna] > limite_superior].tolist())
    print(f"Outliers na coluna {coluna}: {outliers_coluna}")



# Visualização com colunas
# for coluna in ['preco', 'frete_valor']:
#     sns.boxplot(x=df[coluna])


# Excluindo os outliers

for coluna in [ 'preco', 'frete_valor']:
    df = df.drop(df[coluna][df[coluna] < limite_inferior].index, axis=0)
    df = df.drop(df[coluna][df[coluna] > limite_superior].index, axis=0)

# Aqui, vemos que o valor não está atualizado de acordo com os calculos acima
#display(df.loc[df['id_pedido'] == '00143d0f86d6fbd9f9b38ab440ac16f5'])

df['preco_total'] = df['preco'] * df['numero_itens']
df['frete_total'] = df['frete_valor'] * df['numero_itens']

# display(df)

# excluindo a coluna

df = df.drop('limite_entrega', axis=1)

# CRIAÇÃO DE UMA COLUNA

df['total'] = df['preco_total'] + df['frete_total']

# display(df.loc[df['numero_itens'] > 1])

# Cria um novo arquivo csv já limpo e salva nesse caminho
df.to_csv(CLEAN_PATH, index=False)


Outliers na coluna preco: [810.0, 639.0, 289.0, 397.0, 809.1, 390.0, 492.75, 399.99, 540.0, 469.0, 499.0, 308.0, 579.0, 449.0, 1050.61, 362.45, 320.0, 849.0, 349.9, 419.9, 299.0, 299.0, 379.0, 593.36, 389.0, 899.0, 501.9, 1820.0, 399.9, 504.0, 610.0, 292.9, 299.0, 379.0, 359.98, 579.99, 399.9, 320.0, 349.99, 738.9, 579.0, 299.99, 309.9, 465.0, 629.9, 359.0, 339.99, 398.0, 329.9, 330.0, 1240.0, 1799.0, 329.0, 559.99, 479.0, 989.0, 550.0, 349.99, 349.9, 509.0, 325.0, 499.9, 827.99, 379.0, 329.0, 2649.99, 467.0, 379.0, 1350.0, 292.9, 319.0, 329.9, 454.0, 301.0, 349.9, 699.0, 388.0, 399.99, 399.99, 479.0, 779.0, 1199.9, 336.0, 630.0, 479.0, 538.9, 1149.65, 339.9, 288.92, 1890.0, 299.99, 314.99, 1999.0, 858.9, 379.9, 467.0, 349.9, 599.0, 1680.0, 499.0, 364.98, 649.0, 297.0, 887.0, 519.0, 599.5, 349.0, 569.9, 489.0, 467.0, 379.9, 478.0, 335.9, 319.0, 599.0, 443.0, 668.0, 329.9, 735.0, 599.0, 439.9, 629.9, 329.9, 499.0, 299.0, 898.9, 695.72, 499.9, 379.99, 397.9, 749.9, 585.0, 1235.99, 388.0,

/home/graziely/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CONTEXTO DA TABELA

essa tabela inclui informações sobre cada pedido feito

EXPLICAÇÃO DAS COLUNAS

ID PEDIDO - id do pedido ( por isso decidi excliuir duplicatas por esse item)

NUMERO_ITENS - numero de itens do pedido, por exemplo, em um pedido eu posso comprar 2 liros e um marca-texto, tendo três itens

ID_PRODUTO - id do produto comprado

ID_VENDEDOR - id do vendedor dos produtos

PRECO - preco do items

FRETE - valor do frete, calculado de acordo com as medidas e o peso

TOTAL - frete + preco